In [1]:
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

# fine tune model

In [ ]:
#yolo settings datasets_dir='/Volumes/Tony SSD/Projekte/Studienarbeit/Datasets/Pith/tree-disk-pith.v2i.yolov12'

In [ ]:
base_model_path = "../models/yolo11s.pt"
dataset_path = "/Volumes/Tony SSD/Projekte/Studienarbeit/Datasets/Pith/tree-disk-pith.v2i.yolov12/data.yaml"

In [ ]:
# check if the dataset exists
if not os.path.exists(dataset_path):
    print("Dataset not found")
    exit()

# check if model exists
if not os.path.exists(base_model_path):
    print("Model not found")
    exit()

In [ ]:
# Load a model
base_model = YOLO(base_model_path)

# Train the model
train_results = base_model.train(
    data=dataset_path,  # path to dataset YAML
    epochs=100,  # number of training epochs
    imgsz=640,  # training image size
    device="cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    batch=32, # batch size
)

In [ ]:
# Evaluate model performance on the validation set
metrics = base_model.val()

# results

## Plot segmented masks

In [ ]:
trained_model_path = "../models/yolo11s-det-pith.pt"

In [ ]:
# check if the model exists
if not os.path.exists(trained_model_path):
    print("Model not found")
    exit()

In [ ]:
trained_model = YOLO(trained_model_path)

In [ ]:
results = trained_model("../input/tree-disk4.jpg")

## Plot segmentation mask with original colors

In [ ]:
for result in results:
    height, width = result.orig_img.shape[:2]
    background = np.ones((height, width, 3), dtype=np.uint8) * 255
    masks = result.masks.xy
    orig_img = result.orig_img

    for mask in masks:
        mask = mask.astype(int)
        mask_img = np.zeros_like(orig_img)

        cv2.fillPoly(mask_img, [mask], (255, 255, 255))
        masked_object = cv2.bitwise_and(orig_img, mask_img)
        background[mask_img == 255] = masked_object[mask_img == 255]

    background_rgb = cv2.cvtColor(background, cv2.COLOR_BGR2RGB)

    plt.imshow(background_rgb)
    plt.title('Segmented objects')
    plt.axis('off')
    plt.show()

    cv2.imwrite('segmented_objects.jpg', background)